## pip install everything

In [1]:
# !sudo yum install Xvfb -y -y
# !pip install scrapy
# !pip install fake_useragent
# !pip install scrapy-selenium
# !pip install selenium==4.9.1
# !pip install boto3==1.28.5
# !pip install crochet
# !pip install awslambdaric
# !pip install botocore==1.31.10
# #

# import os
# cmd = '''
# sudo yum install atk cups-libs gtk3 libXcomposite alsa-lib libXcursor libXdamage libXext libXi libXrandr libXScrnSaver libXtst pango at-spi2-atk libXt xorg-x11-server-Xvfb xorg-x11-xauth dbus-glib dbus-glib-devel -y
# '''
# os.system(cmd)


# !curl  -Lo "/tmp/chromedriver.zip" "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"
# !unzip /tmp/chromedriver.zip -d /opt/ -y
# !sudo ln -sf /opt/chromedriver /usr/local/bin/chromedriver

# !curl -Lo "/tmp/chrome-linux.zip" "https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F1135561%2Fchrome-linux.zip?alt=media"
# !unzip /tmp/chrome-linux.zip -d /opt/ -y
# !sudo ln -sf /opt/chrome-linux/chrome /usr/local/bin/chrome



### In terminal 

$sudo chmod 777 /etc/yum.repos.d/docker-ce.repo

$vi /etc/yum.repos.d/docker-ce.repo


#### [file]
1. name=Docker CE Stable - $basearch

    ##baseurl=https://download.docker.com/linux/centos/$releasever/$basearch/stable
            
2. baseurl=https://download.docker.com/linux/centos/7/$basearch/stable

In [2]:
!ls -al /opt/chrome-linux/chrome
!ls -al /usr/local/bin/chrome
!ls -al /opt/chromedriver 
!ls -al /usr/local/bin/chromedriver

-rwxr-xr-x 1 ec2-user ec2-user 358375680 Apr 25 22:16 /opt/chrome-linux/chrome
lrwxrwxrwx 1 root root 24 Aug 25 02:38 /usr/local/bin/chrome -> /opt/chrome-linux/chrome
ls: cannot access /opt/chromedriver: No such file or directory
lrwxrwxrwx 1 root root 17 Aug 25 02:38 /usr/local/bin/chromedriver -> /opt/chromedriver


In [3]:
# !/usr/local/bin/chrome --headless --no-sandbox --disable-gpu --remote-debugging-port=9222
# !/usr/local/bin/chromedriver

## tier2

In [4]:
import json
import os, datetime, requests
import botocore
from fake_useragent import UserAgent
import boto3
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from crochet import setup
import threading
from momoshop import MomoshopSpider
# from momoshop_tier4 import MomoshopSpider as MomoshopSpider_tier4

# Initialize Crochet
setup()

ua = UserAgent()
region = os.getenv('ap-northeast-1')
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
queue_tier2_links = os.getenv('https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier2_links.fifo')

In [5]:
class LambdaRunner:
    category_links = []

    def __init__(self):
        self.finished = threading.Event()
        self.results = []

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)

        # Create an instance of the spider class
        spider_cls = MomoshopSpider

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(spider_cls, self)
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    def wait_for_completion(self):
        self.finished.wait()

    def get_results(self):
        return self.results
    
    

def handler(event, context):
    try:
        print("Starting tier2 links crawling")
        runner = LambdaRunner()
        runner.run_spider()
        runner.wait_for_completion()

        queue_list = []

        # Aggregate input JSON format
        for category_link in runner.category_links:
            queue_list.append({'category_link': category_link})
        print(f'len(queue_list): {len(runner.category_links)}')
        print("success")
        return {
            'statusCode': 200,
            'body': {'message': queue_list}
        }
    except Exception as e:
        print(f'fail:{str(e)}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

In [6]:
# handler("", "")

## tier 4 

In [7]:
import hashlib
import json
import threading
import scrapy
from fake_useragent import UserAgent
from scrapy import Selector
from scrapy.http import HtmlResponse
from scrapy_selenium import SeleniumRequest
import os
import boto3
import botocore
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from urllib.parse import urlparse


# Initialize the UserAgent and AWS clients
ua = UserAgent()
region = 'ap-northeast-1'
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
s3 = boto3.client('s3', region_name=region, config=botocore.client.Config(max_pool_connections=500))
queue_tier2_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier2_links.fifo'
queue_tier4_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'
s3_bucket_name = 's3://aws-web-crawler-data/PIC_share/'
export_to_s3 = 'Off'


# Function to determine the tier based on the URL
def check_tier_by_url(url: str):
    if url.find("DgrpCategory") != -1:
        return "tier4"
    elif url.find("MgrpCategory") != -1:
        return "tier3"
    elif url.find("LgrpCategory") != -1:
        return "tier2"
    else:
        return None


# Function to calculate SHA-256 hash of a string
def hash_function(original_string):
    # Create a hash object using SHA-256 algorithm
    hasher = hashlib.sha256()

    # Convert the input string to bytes (required by hashlib)
    input_bytes = original_string.encode('utf-8')

    # Update the hash object with the input bytes
    hasher.update(input_bytes)

    # Get the hexadecimal representation of the hash value
    return hasher.hexdigest()


# Spider class for crawling momoshop.com.tw
class MomoshopSpider(scrapy.Spider):
    name = "momoshop"
    allowed_domains = ["www.momoshop.com.tw"]
    user_agent = ua.random
    receipt_handle = ""
    target_url = ""
    runner = None

    def __init__(self, url=None, runner=None, **kwargs):
        self.target_url = url
        self.runner = runner
        super(MomoshopSpider, self).__init__(**kwargs)


    # Function to handle errors during parsing
    def error_handle(self, error):
        print(error)
        print(f'Invalid page: {self.target_url}')
        self.runner.timeout = True

    # Entry point for the spider
    def start_requests(self):
        print("Starting queue_tier2_links from start_requests")
        print(f'self.target_url: {self.target_url}')
        yield SeleniumRequest(url=self.target_url,
                              callback=self.parse_tier2,
                              wait_time=15,
                              errback=self.error_handle,
                              wait_until=ec.any_of(ec.presence_of_element_located((By.CLASS_NAME, 'first')),  # Wait for the tier path loaded
                                                   ec.presence_of_element_located((By.ID, 'backgroundContent')),  # Wait for the landing page loaded
                                                   ec.presence_of_element_located((By.CLASS_NAME, "year18Ban"))))  # Wait for the yes18years paged loaded

#     # Function to send a message to SQS
#     @staticmethod
#     def send_to_sqs(message):
#         try:
#             # print(message['MessageGroupId'])
#             response = sqs.send_message(QueueUrl=queue_tier4_links,
#                                         MessageGroupId=message['MessageGroupId'],
#                                         MessageBody=message['MessageBody'],
#                                         MessageDeduplicationId=message['MessageGroupId'])
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

#     # Function to send data to S3
#     @staticmethod
#     def send_to_s3(m, hash_id):
#         try:
#             json_file_name = f'{hash_id}.json'
#             response = s3.put_object(Bucket=s3_bucket_name, Key=json_file_name, Body=json.dumps(m, ensure_ascii=False))
#             # check if it's successful
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

    # Function to start pushing data to SQS and S3
    def start_to_push(self, current_tier, hash_id, tier_dict, category_link, message_body):
        if current_tier == "tier4":
            print(f'tier4: {tier_dict.get("tier4", "")}')
            message = {'Id': hash_id, 'MessageGroupId': hash_id,
                       'MessageBody': json.dumps({
                           'tier1_category_name': tier_dict["tier1"],
                           'tier2_category_name': tier_dict.get("tier2", ""),
                           'tier3_category_name': tier_dict.get("tier3", ""),
                           'tier4_category_name': tier_dict.get("tier4", ""),
                           'category_link': category_link if category_link.find(
                               "javascript") == -1 else "",
                           'category_type': current_tier}, ensure_ascii=False)}
            print(message)
            # self.send_to_sqs(message)
            # self.runner.category_info.append(message_body)

            # if export_to_s3 is not None and export_to_s3 == "On":
                # self.send_to_s3(message_body, hash_id)

    # Function to parse the tier 2 categories
    def parse_tier2(self, response: HtmlResponse):
        try:

            print("Starting to parse_tier2")
            
            #-------------- add to prevent bugs from jupyter-lab --------------------
            options = webdriver.ChromeOptions()
            options.binary_location = "/opt/chrome-linux/chrome"
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--single-process")
            options.add_argument("blink-settings=imagesEnabled=false")
            options.add_argument("--remote-debugging-port=9222")
            options.add_argument("--no-zygote")
            options.add_argument("--disable-gpu")
            options.add_argument("--disable-dev-tools")
            options.add_argument("--disable-dev-shm-usage")

            driver = webdriver.Chrome(executable_path = '/usr/local/bin/chromedriver', chrome_options=options)
            driver.get(response.url)

            wait = WebDriverWait(driver, 20)  # Adjust the timeout as needed
            wait.until(ec.any_of(ec.presence_of_element_located((By.CLASS_NAME, 'first')),  # Wait for the tier path loaded
                               ec.presence_of_element_located((By.ID, 'backgroundContent')),  # Wait for the landing page loaded
                               ec.presence_of_element_located((By.CLASS_NAME, "year18Ban"))))  # Wait for the yes18years paged loaded

            updated_content = driver.page_source

            response = scrapy.http.HtmlResponse(url=driver.current_url, 
                                                body=updated_content,
                                                encoding='utf-8')

            #----------------------------------------------------------------
            print(driver.current_url)
            # Check yes18years page and click it to go to the next page
            if response.url.find("AgeCheck") != -1:

                # driver = response.request.meta['driver']

                js_code = "document.querySelector(\".yes18years\").click();"  # Click to the next page
                driver.execute_script(js_code)
                
                # Wait for the new content to load after the click
                # Adjust the timeout as needed
                wait = WebDriverWait(driver, 20)
                # Wait for goods comment is ready for click
                wait.until(ec.presence_of_element_located((By.XPATH, "//*")))
                
                # Get the updated page source and extract product links
                updated_content = driver.page_source
                response = Selector(scrapy.http.HtmlResponse(url=driver.current_url,
                                                             body=updated_content,
                                                             encoding='utf-8'))

            


            categories = response.css('#bt_category_Content ul li')

            tier_dict = {}
            threads = []
            if len(categories) > 0:
                tier_path = response.css("#bt_2_layout_NAV") 

                tier_index = 1
                for tier in tier_path:
                    tier_temp_path = tier.css('ul li') 

                    if tier_temp_path.css("::attr(class)").get() != 'first':
                        tier_a = tier_temp_path.css("a").get()
                        if tier_a is not None:
                            tier_dict[f'tier{tier_index}'] = tier_temp_path.css("a::text").get()
                        else:
                            tier_dict[f'tier{tier_index}'] = tier_temp_path.css("::text").get()
                        tier_index = tier_index+1
            print(tier_dict)
            if "tier1" in tier_dict:
                for category in categories:
                    self.parse_process(category, tier_dict)
                    # thread = threading.Thread(target=self.parse_process, args=(category, tier_dict,))
                    # threads.append(thread)
                    # thread.start()
                # for thread in threads:
                    # thread.join()
        except Exception as e:
            print(e)
        finally:
            self.runner.timeout = False

    # Function to process individual category items
    def parse_process(self, category, tier_dict):

        category_item_a = category.css('a').get()
        category_link = ""
        if category_item_a is not None:
            category_link = category.css('li a::attr(href)').get()

        current_tier = check_tier_by_url(category_link)

        if current_tier == "tier4":
            tier_dict['tier4'] = category.css("a::text").get()
        elif current_tier == "tier3":
            tier_dict['tier3'] = category.css("a::text").get()

        if tier_dict is not None:
            category_link = "https://www.momoshop.com.tw" + category_link if category_link.find("momoshop.com.tw") == -1 else category_link
            message_body = {'tier1_category_name': tier_dict["tier1"],
                            'tier2_category_name': tier_dict.get("tier2", ""),
                            'tier3_category_name': tier_dict.get("tier3", ""),
                            'tier4_category_name': tier_dict.get("tier4", ""),
                            # https://ecm.momoshop.com.tw/category/DgrpCategory.jsp?d_code=1704300019&p_orderType=6&showType=chessboardType&sourcePageType=4
                            'category_link': category_link.replace("ecm.momoshop.com.tw", "www.momoshop.com.tw"),
                            'category_type': current_tier}

            hash_id = hash_function(str(category_link))
            self.start_to_push(current_tier, hash_id, tier_dict, category_link, message_body)


In [8]:
#============================
import datetime
import os
import boto3
import botocore
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from crochet import setup
import json
import threading
import momocrawler.settings
import momocrawler.middlewares
import momocrawler.pipelines
# from momoshop_tier4 import MomoshopSpider

# Retrieve environment variables
queue_tier2_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier2_links.fifo'
region = 'ap-northeast-1'
# sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
timeout_seconds = float(os.getenv('TIMEOUT')) if os.getenv('TIMEOUT') is not None else 150  # seconds
# Initialize Crochet
setup()


# Class for running the Lambda spider
class LambdaRunner:
    target_url = ""
    receipt_handle = ""
    timeout = True
    input_url = ""
    category_info = []

    def __init__(self, url):
        self.finished = threading.Event()
        self.results = []
        if url != "":
            self.target_url = url

    # Function to run the spider
    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)

        # Create an instance of the spider class
        spider_cls = MomoshopSpider

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(spider_cls, url=self.target_url, runner=self)
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    # Function to wait for spider completion
    def wait_for_completion(self):
        self.finished.wait(timeout=timeout_seconds)

    # Function to get spider results
    def get_results(self):
        return self.results


# Main handler function for the Lambda
def handler(event, context):
    try:
        print(f"Starting to crawl: {datetime.datetime.now()}")
        times = 0
        if "statusCode" not in event:
            # Initialize the LambdaRunner with the provided category link

            # headers = {"Content-Type": "application/x-www-form-urlencoded",
            #            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
            # response = requests.get('https://myip.com.tw/', headers=headers)
            #
            # # Check the response status code
            # if response.status_code == 200:
            #     ip = response.text[response.text.find("<font color=green>") + 18:response.text.find("</font></h1>")]
            #     # Request was successful
            #     print(ip)
            # else:
            #     # Request failed
            #     print('Request failed with status code:', response.status_code)
            runner = LambdaRunner(event["category_link"])
            runner.input_url = event["category_link"]
            runner.run_spider()
            runner.wait_for_completion()

            print(f"End date and time:{datetime.datetime.now()}")

        else:
            times = int(event["times"])
            if times < 4:
                # Initialize the LambdaRunner with the provided category link
                runner = LambdaRunner(event["category_link"])
                runner.input_url = event["category_link"]
                runner.run_spider()
                runner.wait_for_completion()
                print(f"End date and time:{datetime.datetime.now()}")
            else:
                return {
                    'statusCode': 429,
                    'body': "",
                    'times': times,
                    "category_link": event["category_link"]
                }

        times = times + 1
        if not runner.timeout:
            return {
                'statusCode': 200,
                'body': 'Completed!',
                'times': times,
                # "category_link": event["category_link"],
                # "category_info": runner.category_info
            }
        else:
            return {
                'statusCode': 408,
                'body': event["category_link"],
                'times': times,
                "category_link": event["category_link"]
            }
    except Exception as e:
        print(e)
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }


In [ ]:
# #normal page test
# handler({'category_link': 'https://www.momoshop.com.tw/category/LgrpCategory.jsp?l_code=4300100000&mdiv=1099700000-bt_0_997_12-bt_0_997_12_P101_2_e1&ctype=B'
#          ,'body': 'https://www.momoshop.com.tw/'},
#         "")


#Agecheck page test

handler({'category_link': 'https://www.momoshop.com.tw/category/LgrpCategory.jsp?l_code=1205200000&mdiv=1099700000-bt_0_997_13-bt_0_997_13_P103_1_e1&ctype=B', 
         'body': 'https://www.momoshop.com.tw/'},
        "")

Starting to crawl: 2023-08-25 09:13:58.111141
Starting queue_tier2_links from start_requests
self.target_url: https://www.momoshop.com.tw/category/LgrpCategory.jsp?l_code=1205200000&mdiv=1099700000-bt_0_997_13-bt_0_997_13_P103_1_e1&ctype=B
Starting to parse_tier2
